In [1]:
print("all_ok")

all_ok


In [2]:
import sys
print(sys.executable)


c:\Users\Admin\Documents\llmops\document_portal\llmops\python.exe


In [3]:
from langchain_groq import ChatGroq



In [4]:
import sys
print(sys.executable)

import langchain, langchain_core
print("langchain:", langchain.__version__)
print("langchain_core:", langchain_core.__version__)

import langchain_groq
print("langchain_groq:", langchain_groq.__version__)

c:\Users\Admin\Documents\llmops\document_portal\llmops\python.exe
langchain: 0.3.15
langchain_core: 0.3.72
langchain_groq: 0.3.6


In [5]:
from dotenv import load_dotenv

load_dotenv()


True

In [6]:

llm=ChatGroq(model="deepseek-r1-distill-llama-70b")



In [7]:


print(llm.invoke("What is the capital of France?").content)



<think>

</think>

The capital of France is Paris.


In [8]:


from langchain_google_genai import GoogleGenerativeAIEmbeddings



In [ ]:


embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")


DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

: 

In [ ]:

doc_vector=embedding_model.embed_query("What is the capital of France?")

1. Data Ingestion

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:

import os
os.getcwd()

file_path=os.path.join(os.getcwd(), "data", "sample.pdf")

In [ ]:
loader=PyPDFLoader(file_path)

In [ ]:
documents=loader.load()

len(documents)

Experimentation

In [ ]:

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

In [ ]:
docs=text_splitter.split_documents(documents)

In [ ]:

len(docs)

In [ ]:

docs[0]


In [ ]:

docs[0].metadata



In [ ]:


docs[1].metadata



In [ ]:


0,1,2,3
page_lable=4
actual page is 4 

page_index=3
page_lable=4



In [ ]:


docs[100].metadata



In [ ]:


docs[200].metadata



In [ ]:


docs[0].page_content



In [ ]:


from langchain.vectorstores import FAISS

docs[0].page_content


In [ ]:
doc_vector=embedding_model.embed_documents(docs[0].page_content)

In [ ]:

len(embedding_model.embed_documents(docs[0].page_content)[0])

In [ ]:
vectorstore=FAISS.from_documents(docs, embedding_model)

In [ ]:
token->words

chunk--> it is collection of words(token)[characters]

In [ ]:


relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.")



In [ ]:

relevant_doc

In [ ]:

relevant_doc[0].page_content

In [ ]:
relevant_doc[1].page_content

In [ ]:


relevant_doc[2].page_content



In [ ]:
relevant_doc[3].page_content

In [ ]:


relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.",k=10)



In [ ]:
relevant_doc

In [ ]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.",k=1)

In [ ]:


relevant_doc



In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
retriever.invoke("llama2 finetuning benchmark experiments.")

Question: user question

Context: based on the question retrieving the info from the vector database

In [ ]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""

In [ ]:


from langchain.prompts import PromptTemplate



In [ ]:

prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)



In [ ]:
prompt

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser=StrOutputParser()

In [ ]:

def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])



In [ ]:

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")

In [ ]:


rag_chain.invoke("can you tell me Scaling trends for the reward model?")

